In [1]:
dna_dic_hg = {}
with open("/home/mouren/Data/final_files_tokeep/sequences_species/ee_dna_seqs.tsv") as file:
    for line in file:           
        dna_dic_hg[line.strip().split()[0]] = line.strip().split()[1].upper()
with open("/home/mouren/Data/final_files_tokeep/sequences_species/ctrlNeg_dna_seqs.tsv") as file:
    for line in file:           
        dna_dic_hg[line.strip().split()[0]] = line.strip().split()[1].upper()

dna_dic_mm = {}
with open("/home/mouren/Data/final_files_tokeep/sequences_species/mmus/mm39_{ee}_dna_seqs.tsv") as file:
    for line in file:           
        dna_dic_mm[line.strip().split()[0]] = line.strip().split()[1].upper()
with open("/home/mouren/Data/final_files_tokeep/sequences_species/mmus/mm39_{neg}_dna_seqs.tsv") as file:
    for line in file:           
        dna_dic_mm[line.strip().split()[0]] = line.strip().split()[1].upper()

dna_dic_dm = {}
with open("/home/mouren/Data/final_files_tokeep/sequences_species/dmel/dm6_{ee}_dna_seqs.tsv") as file:
    for line in file:           
        dna_dic_dm[line.strip().split()[0]] = line.strip().split()[1].upper()
with open("/home/mouren/Data/final_files_tokeep/sequences_species/dmel/dm6_{neg}_dna_seqs.tsv") as file:
    for line in file:           
        dna_dic_dm[line.strip().split()[0]] = line.strip().split()[1].upper()

dna_dic_tair = {}
with open("/home/mouren/Data/final_files_tokeep/sequences_species/thal/tair10_{ee}_dna_seqs.tsv") as file:
    for line in file:           
        dna_dic_tair[line.strip().split()[0]] = line.strip().split()[1].upper()
with open("/home/mouren/Data/final_files_tokeep/sequences_species/thal/tair10_{neg}_dna_seqs.tsv") as file:
    for line in file:           
        dna_dic_tair[line.strip().split()[0]] = line.strip().split()[1].upper()

In [2]:
### Functions
def reverse_complement(seq):
    """Return the reverse complement of a DNA sequence."""
    # Define the complement mapping for nucleotides
    complement = {
        'A': 'T', 'T': 'A',
        'C': 'G', 'G': 'C',
        'a': 't', 't': 'a',
        'c': 'g', 'g': 'c',
        # Optionally handle ambiguous bases
        'N': 'N', 'n': 'n'
    }
    
    # Build the complement sequence
    comp_seq = []
    for base in seq:
        if base in complement:
            comp_seq.append(complement[base])
        else:
            # If the base is not recognized, preserve it or handle as needed
            comp_seq.append(base)
    
    # Reverse the complemented sequence
    rev_comp = ''.join(comp_seq)[::-1]
    return rev_comp

def levenshtein_distance(s, t):
    """Compute the Levenshtein distance between two strings."""
    if s == t:
        return 0
    if len(s) == 0:
        return len(t)
    if len(t) == 0:
        return len(s)

    # Initialize distance matrix
    previous_row = list(range(len(t) + 1))

    for i, sc in enumerate(s, start=1):
        current_row = [i]
        for j, tc in enumerate(t, start=1):
            # Calculate costs for deletion, insertion, substitution
            deletion    = previous_row[j] + 1
            insertion   = current_row[j - 1] + 1
            substitution = previous_row[j - 1] + (sc != tc)
            current_row.append(min(deletion, insertion, substitution))
        previous_row = current_row

    return previous_row[-1]

def string_similarity(str1, str2):
    """
    Compute a similarity score between 0 and 1 based on the normalized Levenshtein distance.
    1.0 means identical strings, 0.0 means completely different.
    """
    max_len = max(len(str1), len(str2))
    # If both strings are empty, they are identical
    if max_len == 0:
        return 1.0
    
    dist = levenshtein_distance(str1, str2)
    # Normalize similarity: 1 - (distance / maximum possible length)
    similarity_score = 1 - (dist / max_len)
    return similarity_score

# Example usage:
# s1 = "kitten"
# s2 = "sitting"
# score = string_similarity(s1, s2)
# print(f"Similarity between '{s1}' and '{s2}' is {score:.2f}")


In [3]:
### HSAP EE
import pandas as pd


df_ee_hg = pd.read_csv("/home/mouren/Data/final_files_tokeep/aa_sequences/results/hg38_ee_aa_seq.tsv", sep="\t", header=0)

grouped_df = df_ee_hg.groupby("name")

res_ee_aa = {}
for group_name, group_data in grouped_df:
    
    dna = dna_dic_hg[group_name]
    if group_data.iloc[0, 3] == "-":
        dna = reverse_complement(dna)
      
    # If there is only one match with gene2proteo result
    if len(group_data) == 1:
        # check if they are the same first to be faster :
        if dna == group_data.iloc[0, 6]:
            res_ee_aa[group_name] = group_data.iloc[0, 9]
            continue
        else: #check similarity
            dna_similarity = string_similarity(dna, group_data.iloc[0, 6])
            if dna_similarity >= 0.9:
                res_ee_aa[group_name] = group_data.iloc[0, 9]
                continue
            else:
                print("solo entry", group_name)
    
    #choose entry with the most similar sequence
    else:
        score_closest_seq = 0
        aa_seq = ""
        for index, row in group_data.iterrows():
            if dna == row["dnaSeq"]:
                score_closest_seq = 1
                aa_seq = row["pepSeq"]
                break
            else:
                score = string_similarity(dna, row["dnaSeq"])
                if score > score_closest_seq:
                    score_closest_seq = score
                    aa_seq = row["pepSeq"]

        if score_closest_seq >= 0.9:
            res_ee_aa[group_name] = aa_seq
        else:
            print(group_name)

with open("/home/mouren/Data/final_files_tokeep/aa_sequences/sequences/ee_aa_seqs_hg38.txt", 'w') as file:
    for key, value in res_ee_aa.items():
        file.write(f'{key}\t{value}\n')

In [ ]:
### HSAP NEG
import pandas as pd


df_neg_hg = pd.read_csv("/home/mouren/Data/final_files_tokeep/aa_sequences/results_tool/hg38_neg_aa_seq.tsv", sep="\t", header=0, keep_default_na=False)

grouped_df = df_neg_hg.groupby("name")

not_found = []
res_neg_aa = {}
for group_name, group_data in grouped_df:
    
    dna = dna_dic_hg[group_name]
    if group_data.iloc[0, 3] == "-":
        dna = reverse_complement(dna)
      
    # If there is only one match with gene2proteo result
    if len(group_data) == 1:
        # check if they are the same first to be faster :
        if dna == group_data.iloc[0, 6]:
            res_neg_aa[group_name] = group_data.iloc[0, 9]
            continue
        else: #check similarity
            try:
                dna_similarity = string_similarity(dna, group_data.iloc[0, 6])
            except TypeError:
                print(group_name)
                break
            if dna_similarity >= 0.9:
                res_neg_aa[group_name] = group_data.iloc[0, 9]
                continue
            else:
                not_found.append(group_name)
                print("solo entry", group_name)
    
    #choose entry with the most similar sequence
    else:
        score_closest_seq = 0
        aa_seq = ""
        for index, row in group_data.iterrows():
            if dna == row["dnaSeq"]:
                score_closest_seq = 1
                aa_seq = row["pepSeq"]
                break
            else:
                score = string_similarity(dna, row["dnaSeq"])
                if score > score_closest_seq:
                    score_closest_seq = score
                    aa_seq = row["pepSeq"]

        if score_closest_seq >= 0.9:
            res_neg_aa[group_name] = aa_seq
        else:
            not_found.append(group_name)
            print(group_name)

for i in not_found:
    res_neg_aa[i] = "false"

with open("/home/mouren/Data/final_files_tokeep/aa_sequences/sequences/neg_aa_seqs_hg38.txt", 'w') as file:
    for key, value in res_neg_aa.items():
        file.write(f'{key}\t{value}\n')

In [9]:
### MM EE
import pandas as pd

df_ee_mm = pd.read_csv("/home/mouren/Data/final_files_tokeep/aa_sequences/results_tool/mm39_ee_aa_seq.tsv", sep="\t", header=0, keep_default_na=False)

grouped_df = df_ee_mm.groupby("name")

not_found = []
res_ee_aa = {}
for group_name, group_data in grouped_df:
    
    dna = dna_dic_mm[group_name]
    if group_data.iloc[0, 3] == "-":
        dna = reverse_complement(dna)
      
    # If there is only one match with gene2proteo result
    if len(group_data) == 1:
        # check if they are the same first to be faster :
        if dna == group_data.iloc[0, 6]:
            res_ee_aa[group_name] = group_data.iloc[0, 9]
            continue
        else: #check similarity
            try:
                dna_similarity = string_similarity(dna, group_data.iloc[0, 6])
            except TypeError:
                print(group_name)
                break
            if dna_similarity >= 0.9:
                res_ee_aa[group_name] = group_data.iloc[0, 9]
                continue
            else:
                not_found.append(group_name)
                print("solo entry", group_name)
    
    #choose entry with the most similar sequence
    else:
        score_closest_seq = 0
        aa_seq = ""
        for index, row in group_data.iterrows():
            if dna == row["dnaSeq"]:
                score_closest_seq = 1
                aa_seq = row["pepSeq"]
                break
            else:
                score = string_similarity(dna, row["dnaSeq"])
                if score > score_closest_seq:
                    score_closest_seq = score
                    aa_seq = row["pepSeq"]

        if score_closest_seq >= 0.9:
            res_ee_aa[group_name] = aa_seq
        else:
            not_found.append(group_name)
            print(group_name)

for i in not_found:
    res_ee_aa[i] = "false"

with open("/home/mouren/Data/final_files_tokeep/aa_sequences/sequences/ee_aa_seqs_mm39.txt", 'w') as file:
    for key, value in res_ee_aa.items():
        file.write(f'{key}\t{value}\n')

In [ ]:
### MM NEG
import pandas as pd


df_ee_mm = pd.read_csv("/home/mouren/Data/final_files_tokeep/aa_sequences/results_tool/mm39_neg_aa_seq.tsv", sep="\t", header=0, keep_default_na=False)

grouped_df = df_ee_mm.groupby("name")

not_found = []
res_ee_aa = {}
for group_name, group_data in grouped_df:
    
    dna = dna_dic_mm[group_name]
    if group_data.iloc[0, 3] == "-":
        dna = reverse_complement(dna)
      
    # If there is only one match with gene2proteo result
    if len(group_data) == 1:
        # check if they are the same first to be faster :
        if dna == group_data.iloc[0, 6]:
            res_ee_aa[group_name] = group_data.iloc[0, 9]
            continue
        else: #check similarity
            try:
                dna_similarity = string_similarity(dna, group_data.iloc[0, 6])
            except TypeError:
                print(group_name)
                break
            if dna_similarity >= 0.9:
                res_ee_aa[group_name] = group_data.iloc[0, 9]
                continue
            else:
                not_found.append(group_name)
                print("solo entry", group_name)
    
    #choose entry with the most similar sequence
    else:
        score_closest_seq = 0
        aa_seq = ""
        for index, row in group_data.iterrows():
            if dna == row["dnaSeq"]:
                score_closest_seq = 1
                aa_seq = row["pepSeq"]
                break
            else:
                score = string_similarity(dna, row["dnaSeq"])
                if score > score_closest_seq:
                    score_closest_seq = score
                    aa_seq = row["pepSeq"]

        if score_closest_seq >= 0.9:
            res_ee_aa[group_name] = aa_seq
        else:
            not_found.append(group_name)
            print(group_name)

for i in not_found:
    res_ee_aa[i] = "false"

with open("/home/mouren/Data/final_files_tokeep/aa_sequences/sequences/neg_aa_seqs_mm39.txt", 'w') as file:
    for key, value in res_ee_aa.items():
        file.write(f'{key}\t{value}\n')

In [3]:
### DM EE
import pandas as pd

df_ee_dm = pd.read_csv("/home/mouren/Data/final_files_tokeep/aa_sequences/results_tool/res_ovlp_ee_aa_seq_dm6.tsv", sep="\t", header=None, keep_default_na=False)

grouped_df = df_ee_dm.groupby(3)

not_found = []
res_ee_aa = {}
for group_name, group_data in grouped_df:
    
    dna = dna_dic_dm[group_name]
    if group_data.iloc[0, 11] == "-":
        dna = reverse_complement(dna)
      
    # If there is only one match with gene2proteo result
    if len(group_data) == 1:
        # check if they are the same first to be faster :
        if dna == group_data.iloc[0, 12]:
            res_ee_aa[group_name] = group_data.iloc[0, 13]
            continue
        else: #check similarity
            try:
                dna_similarity = string_similarity(dna, group_data.iloc[0, 12])
            except TypeError as e:
                print(e)
                print(group_name)
                break
            if dna_similarity >= 0.9:
                res_ee_aa[group_name] = group_data.iloc[0, 13]
                continue
            else:
                not_found.append(group_name)
                print("solo entry", group_name)
    
    #choose entry with the most similar sequence
    else:
        score_closest_seq = 0
        aa_seq = ""
        for index, row in group_data.iterrows():
            if dna == row[12]:
                score_closest_seq = 1
                aa_seq = row[13]
                break
            else:
                score = string_similarity(dna, row[12])
                if score > score_closest_seq:
                    score_closest_seq = score
                    aa_seq = row[13]

        if score_closest_seq >= 0.9:
            res_ee_aa[group_name] = aa_seq
        else:
            not_found.append(group_name)
            print(group_name)

for i in not_found:
    res_ee_aa[i] = "false"

with open("/home/mouren/Data/final_files_tokeep/aa_sequences/sequences/ee_aa_seqs_dm6.txt", 'w') as file:
    for key, value in res_ee_aa.items():
        file.write(f'{key}\t{value}\n')

FBtr0445314_cds_3_0_chr3R_28835696_r


In [4]:
### DM NEG
import pandas as pd

df_neg_dm = pd.read_csv("/home/mouren/Data/final_files_tokeep/aa_sequences/results_tool/res_ovlp_neg_aa_seq_dm6.tsv", sep="\t", header=None, keep_default_na=False)

grouped_df = df_neg_dm.groupby(3)

not_found = []
res_ee_aa = {}
for group_name, group_data in grouped_df:
    
    dna = dna_dic_dm[group_name]
    if group_data.iloc[0, 11] == "-":
        dna = reverse_complement(dna)
      
    # If there is only one match with gene2proteo result
    if len(group_data) == 1:
        # check if they are the same first to be faster :
        if dna == group_data.iloc[0, 12]:
            res_ee_aa[group_name] = group_data.iloc[0, 13]
            continue
        else: #check similarity
            try:
                dna_similarity = string_similarity(dna, group_data.iloc[0, 12])
            except TypeError as e:
                print(e)
                print(group_name)
                break
            if dna_similarity >= 0.9:
                res_ee_aa[group_name] = group_data.iloc[0, 13]
                continue
            else:
                not_found.append(group_name)
                print("solo entry", group_name)
    
    #choose entry with the most similar sequence
    else:
        score_closest_seq = 0
        aa_seq = ""
        for index, row in group_data.iterrows():
            if dna == row[12]:
                score_closest_seq = 1
                aa_seq = row[13]
                break
            else:
                score = string_similarity(dna, row[12])
                if score > score_closest_seq:
                    score_closest_seq = score
                    aa_seq = row[13]

        if score_closest_seq >= 0.9:
            res_ee_aa[group_name] = aa_seq
        else:
            not_found.append(group_name)
            print(group_name)

for i in not_found:
    res_ee_aa[i] = "false"

with open("/home/mouren/Data/final_files_tokeep/aa_sequences/sequences/neg_aa_seqs_dm6.txt", 'w') as file:
    for key, value in res_ee_aa.items():
        file.write(f'{key}\t{value}\n')

FBtr0072506_cds_4_0_chr3L_274867_f
solo entry FBtr0081580_cds_2_0_chr3R_7287489_f
solo entry FBtr0084894_cds_5_0_chr3R_25592372_r
solo entry FBtr0087219_cds_7_0_chr2R_16114940_f
solo entry FBtr0087293_cds_3_0_chr2R_15938255_r
solo entry FBtr0113148_cds_6_0_chr3L_5916917_f
solo entry FBtr0113679_cds_2_0_chr3L_27856703_r
solo entry FBtr0300847_cds_2_0_chr2L_15551097_f
solo entry FBtr0302310_cds_6_0_chr3L_1633790_r
FBtr0304719_cds_4_0_chr2L_13962120_f
solo entry FBtr0306920_cds_3_0_chr2L_18671782_r
solo entry FBtr0308311_cds_6_0_chr3L_3289885_f
solo entry FBtr0308634_cds_24_0_chr3R_26603560_f
solo entry FBtr0331500_cds_3_0_chr2R_24125565_r
FBtr0331924_cds_8_0_chr2R_10244876_f
FBtr0333109_cds_10_0_chr3L_8563979_f
solo entry FBtr0333493_cds_9_0_chr3L_985726_f
FBtr0334940_cds_2_0_chr3L_27844591_r
solo entry FBtr0335486_cds_4_0_chr2L_7615576_f
solo entry FBtr0336775_cds_13_0_chr3R_16566203_f
solo entry FBtr0339092_cds_5_0_chr2R_21209972_f
solo entry FBtr0339941_cds_3_0_chr2R_22764871_f
FBtr03

In [5]:
### TAIR EE
import pandas as pd

df_ee_tair = pd.read_csv("/home/mouren/Data/final_files_tokeep/aa_sequences/results_tool/res_ovlp_ee_aa_seq_tair10.tsv", sep="\t", header=None, keep_default_na=False)

grouped_df = df_ee_tair.groupby(3)

not_found = []
res_ee_aa = {}
for group_name, group_data in grouped_df:
    
    dna = dna_dic_tair[group_name]
    if group_data.iloc[0, 11] == "-":
        dna = reverse_complement(dna)
      
    # If there is only one match with gene2proteo result
    if len(group_data) == 1:
        # check if they are the same first to be faster :
        if dna == group_data.iloc[0, 12]:
            res_ee_aa[group_name] = group_data.iloc[0, 13]
            continue
        else: #check similarity
            try:
                dna_similarity = string_similarity(dna, group_data.iloc[0, 12])
            except TypeError as e:
                print(e)
                print(group_name)
                break
            if dna_similarity >= 0.9:
                res_ee_aa[group_name] = group_data.iloc[0, 13]
                continue
            else:
                not_found.append(group_name)
                print("solo entry", group_name)
    
    #choose entry with the most similar sequence
    else:
        score_closest_seq = 0
        aa_seq = ""
        for index, row in group_data.iterrows():
            if dna == row[12]:
                score_closest_seq = 1
                aa_seq = row[13]
                break
            else:
                score = string_similarity(dna, row[12])
                if score > score_closest_seq:
                    score_closest_seq = score
                    aa_seq = row[13]

        if score_closest_seq >= 0.9:
            res_ee_aa[group_name] = aa_seq
        else:
            not_found.append(group_name)
            print(group_name)

for i in not_found:
    res_ee_aa[i] = "false"

with open("/home/mouren/Data/final_files_tokeep/aa_sequences/sequences/ee_aa_seqs_tair10.txt", 'w') as file:
    for key, value in res_ee_aa.items():
        file.write(f'{key}\t{value}\n')

AT1G14750.1_cds_6_0_chr1_5079674_r
solo entry AT5G11240.1_cds_13_0_chr5_3586760_f


In [6]:
### TAIR NEG
import pandas as pd

df_neg_tair = pd.read_csv("/home/mouren/Data/final_files_tokeep/aa_sequences/results_tool/res_ovlp_neg_aa_seq_tair10.tsv", sep="\t", header=None, keep_default_na=False)

grouped_df = df_neg_tair.groupby(3)

not_found = []
res_ee_aa = {}
for group_name, group_data in grouped_df:
    
    dna = dna_dic_tair[group_name]
    if group_data.iloc[0, 11] == "-":
        dna = reverse_complement(dna)
      
    # If there is only one match with gene2proteo result
    if len(group_data) == 1:
        # check if they are the same first to be faster :
        if dna == group_data.iloc[0, 12]:
            res_ee_aa[group_name] = group_data.iloc[0, 13]
            continue
        else: #check similarity
            try:
                dna_similarity = string_similarity(dna, group_data.iloc[0, 12])
            except TypeError as e:
                print(e)
                print(group_name)
                break
            if dna_similarity >= 0.9:
                res_ee_aa[group_name] = group_data.iloc[0, 13]
                continue
            else:
                not_found.append(group_name)
                print("solo entry", group_name)
    
    #choose entry with the most similar sequence
    else:
        score_closest_seq = 0
        aa_seq = ""
        for index, row in group_data.iterrows():
            if dna == row[12]:
                score_closest_seq = 1
                aa_seq = row[13]
                break
            else:
                score = string_similarity(dna, row[12])
                if score > score_closest_seq:
                    score_closest_seq = score
                    aa_seq = row[13]

        if score_closest_seq >= 0.9:
            res_ee_aa[group_name] = aa_seq
        else:
            not_found.append(group_name)
            print(group_name)

for i in not_found:
    res_ee_aa[i] = "false"

with open("/home/mouren/Data/final_files_tokeep/aa_sequences/sequences/neg_aa_seqs_tair10.txt", 'w') as file:
    for key, value in res_ee_aa.items():
        file.write(f'{key}\t{value}\n')

solo entry AT1G09150.1_cds_8_0_chr1_2948891_f
AT1G11950.2_cds_11_0_chr1_4035023_r
solo entry AT1G13020.1_cds_2_0_chr1_4440927_r
solo entry AT1G17400.1_cds_4_0_chr1_5960920_r
AT1G22200.2_cds_11_0_chr1_7838104_r
AT1G24793.2_cds_5_0_chr1_8768903_f
AT1G25210.1_cds_5_0_chr1_8834567_f
AT1G27180.3_cds_4_0_chr1_9444901_f
solo entry AT1G30972.2_cds_1_0_chr1_11045300_f
solo entry AT1G30974.2_cds_1_0_chr1_11047659_f
AT1G34120.4_cds_8_0_chr1_12429107_f
solo entry AT1G34460.5_cds_10_0_chr1_12599825_f
solo entry AT1G34770.5_cds_4_0_chr1_12747748_f
solo entry AT1G40087.1_cds_7_0_chr1_14999762_r
solo entry AT1G48880.1_cds_3_0_chr1_18082642_f
solo entry AT1G49940.3_cds_9_0_chr1_18488894_r
solo entry AT1G52870.2_cds_5_0_chr1_19687673_f
AT1G58807.2_cds_4_0_chr1_21783378_f
AT1G59124.1_cds_4_0_chr1_21819636_f
solo entry AT1G61190.2_cds_1_0_chr1_22560420_f
solo entry AT1G63230.1_cds_1_0_chr1_23452178_f
solo entry AT1G63680.1_cds_4_0_chr1_23617229_f
solo entry AT1G78680.2_cds_9_0_chr1_29595959_f
AT2G04845.2_